Mounting google drive, where the data is saved

In [27]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

## Load data

In [30]:
# Load DataFrame back from HDF5 file
df = pd.read_hdf('/content/drive/My Drive/AI4EIC_hackathon/data_AI4EIC_Q1.h5', key='data_new')

df.head()

,eventID,PID,momentum,theta,phi,image
0,0.0,1,15.0,19.999999,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1.0,1,15.0,19.999999,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2.0,1,15.0,19.999999,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3.0,0,15.0,19.999999,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,4.0,0,15.0,19.999999,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [31]:

length = len(df)

images = np.stack(df['image'].to_numpy()).reshape(length,128,128)
print(images.shape)

labels = np.array(df["PID"]).reshape(length,1)
print(labels.shape)


(1000, 128, 128)
(1000, 1)


## Train Test Split

In [32]:

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=421)



## CNN Model

In [33]:
num_filters = 8
filter_size = 3
pool_size = 2

model = Sequential([
    Conv2D(num_filters, filter_size, input_shape=(128, 128, 1)),
    MaxPooling2D(pool_size=pool_size),
    Flatten(),
    Dense(1, activation='sigmoid'),
    ])



model.compile(
  'adam',
  loss='binary_crossentropy',
  metrics=['accuracy'],
)

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 126, 126, 8)       80        
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 63, 63, 8)         0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 31752)             0         
                                                                 
 dense_5 (Dense)             (None, 1)                 31753     
                                                                 
Total params: 31833 (124.35 KB)
Trainable params: 31833 (124.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Traning the Model

In [34]:
model.fit(X_train, y_train, epochs=3, validation_data=( X_test, y_test))

Epoch 1/3
29/29 [==============================] - 2s 18ms/step - loss: 0.5570 - accuracy: 0.7978 - val_loss: 0.3935 - val_accuracy: 1.0000
Epoch 2/3
29/29 [==============================] - 0s 9ms/step - loss: 0.2603 - accuracy: 0.9956 - val_loss: 0.1534 - val_accuracy: 1.0000
Epoch 3/3
29/29 [==============================] - 0s 8ms/step - loss: 0.1048 - accuracy: 0.9956 - val_loss: 0.0660 - val_accuracy: 1.0000


In [36]:
# Save the model to disk.
model.save_weights('/content/drive/My Drive/AI4EIC_hackathon/Q1/AI4EIC_Q1_simple_CNN_weights.h5')
model.save('/content/drive/My Drive/AI4EIC_hackathon/Q1/AI4EIC_Q1_simple_CNN_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
